In [7]:
import pandas as pd

# Load the files
for group in range (1,11):
    oracle_path = f'Group_{group:02}_Oracle (1).csv'
    dped_path = f'../Group_{group:02}_DPed.csv'

    oracle_df = pd.read_csv(oracle_path)
    dped_df = pd.read_csv(dped_path)

    # Merge the two dataframes based on the 'Utterance' column
    # Keeping all columns from oracle_df and only the 'DPed' column from dped_df
    merged_df = pd.merge(oracle_df, dped_df[['Utterance', 'DPed']], on='Utterance', how='left')

    # Only update the 'Transcript' where 'DPed' is not NaN
    merged_df.loc[~merged_df['DPed'].isna(), 'Transcript'] = merged_df['DPed']

    # Drop the now redundant 'DPed' column
    merged_df.drop(columns=['DPed'], inplace=True)

    # Save the updated dataframe to a new CSV file
    updated_file_path = f'Group_{group:02}_Oracle.csv'
    merged_df.to_csv(updated_file_path, index=False)


In [9]:
import pandas as pd
import re
# Function to find overlap between two intervals
def is_overlap(start1, end1, start2, end2):
    return max(start1, start2) < min(end1, end2)


#The following will find the greatest overlap betwween the CGA segments and the oracle segments  
#If there are multiple overlapping segments, then the oracle with the longer transcript will be chosen
#The final output will contain the CGA labels with its corresponding transcript 
for group in range(1, 11):
    print(f"Processing Group {group}")
    # Load the Oracle and CGA CSV files
    oracle_path = f"Group_{group:02}_Oracle.csv" # We just created it in the above cell 
    cga_path = f"Group_{group:02}_CGA (1).csv"  # Corrected path for CGA file

    oracle_df = pd.read_csv(oracle_path)
    cga_df = pd.read_csv(cga_path)

    # Convert time stamps to float
    oracle_df['Start'] = oracle_df['Start'].astype(float)
    oracle_df['End'] = oracle_df['End'].astype(float)
    cga_df['Begin Time - ss.msec'] = cga_df['Begin Time - ss.msec'].astype(float)
    cga_df['End Time - ss.msec'] = cga_df['End Time - ss.msec'].astype(float)

    # Create a new column in CGA for each Oracle column to be merged
    for col in oracle_df.columns:
        if col not in cga_df:
            cga_df[col] = None

    # Iterate through each row in CGA and check for overlap with Oracle rows
    for index, cga_row in cga_df.iterrows():
        cga_start = cga_row['Begin Time - ss.msec']
        cga_end = cga_row['End Time - ss.msec']
        best_overlap_row = None
        max_transcript_length = -1

        for _, oracle_row in oracle_df.iterrows():
            oracle_start = oracle_row['Start']
            oracle_end = oracle_row['End']

            # Check for overlap
            if is_overlap(cga_start, cga_end, oracle_start, oracle_end):
                transcript_length = len(oracle_row['Transcript'])
                
                # Select the row with the longer transcript
                if transcript_length > max_transcript_length:
                    best_overlap_row = oracle_row
                    max_transcript_length = transcript_length

        # If an overlapping row is found, merge it into CGA row
        if best_overlap_row is not None:
            for col in oracle_df.columns:
                cga_df.at[index, col] = best_overlap_row[col]

    # Save the modified CGA dataframe to a new CSV file
    modified_cga_path = f'Golden_Group_{group:02}_CGA.csv'
    cga_df['Common Ground'] = cga_df['Common Ground'].astype(str)

    # Filtering rows where 'Common Ground' contains 'STATEMENT'
    statement_rows = cga_df[cga_df['Common Ground'].str.contains('STATEMENT')]
    print(statement_rows)

    # Define a function to extract content within brackets
    def extract_brackets(text):
        match = re.search(r'\((.*?)\)', text)
        if match:
            return match.group(1)
        else:
            return "No Match Found"  # Return a placeholder if no match is found

    def replace_symbols(text):
        text.replace("equals","=").replace("does not equal","!=").\
                        replace("is less than","<").replace("is more than",">").\
                        replace("plus","+").replace("ten","10").replace("twenty","20").replace("thirty","30").\
                        replace("forty","40").replace("fifty","50").replace("block","").replace(" and ", ",").\
                        replace(" ","")
    # Apply the function to the 'Common Ground' column
    statement_rows.loc[:, 'Common Ground'] = statement_rows['Common Ground'].apply(extract_brackets)
    print(statement_rows)
    statement_rows.to_csv(modified_cga_path)


Processing Group 1


    Begin Time - ss.msec  End Time - ss.msec  \
0                  24.42               27.72   
2                  80.52               81.67   
5                  95.19               97.17   
7                  99.78              102.03   
12                112.42              112.72   
13                112.74              113.76   
16                127.35              129.19   
18                129.47              131.98   
22                140.73              142.81   
29                181.08              182.28   
35                207.06              210.87   
40                238.04              239.92   
45                265.50              268.13   

                                        Common Ground Utterance   Start  \
0                             S0: STATEMENT(red = 10)         5   24.42   
2                             S1: STATEMENT(red = 10)        24   78.08   
5                           S2: STATEMENT(red = blue)        30   95.19   
7                          